## 0.1 Libraries

In [ ]:
#lifelines
from lifelines import CoxPHFitter, WeibullAFTFitter, LogNormalAFTFitter
from lifelines.utils import concordance_index

#pycox
from pycox.models import DeepHitSingle

#pytorch
import torch
import torchtuples as tt

#sklearn
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.calibration import calibration_curve

#others
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import scipy.stats as stats
from hyperopt import fmin, tpe, hp, Trials
from category_encoders import CatBoostEncoder
from dateutil.relativedelta import relativedelta

In [ ]:
#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## 0.2 Function definition

In [ ]:
def missing_zero_values_table(df):
    # Calculate 0 and missing values for every variable of input pandas df
    zero_val = (df == 0.00).astype(int).sum(axis=0)
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
    mz_table = mz_table.rename(
    columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
    mz_table['Data Type'] = df.dtypes
    mz_table = mz_table[
        mz_table.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
        "There are " + str(mz_table.shape[0]) +
        " columns that have missing values.")
    return mz_table
    
def preprocessing(sample):
    # Select catrgorical variables
    cats = [c for c in sample.columns if sample[c].dtypes == 'object']
    # Catboost encoder for categorical variables
    CBE_encoder = CatBoostEncoder()
    sample[cats] = CBE_encoder.fit_transform(sample[cats], sample['default_adj'])
    # Linear interpolation for missing values
    sample = sample.interpolate(method='linear', limit_direction='both')
    # Min-max scaling of variables
    scaler = MinMaxScaler()
    sample[list(set(sample.columns).difference(['intodefault_12', 'default_adj', 'duration', 'cid']))] = scaler.fit_transform(
        sample[list(set(sample.columns).difference(['intodefault_12', 'default_adj', 'duration', 'cid']))]
    )
    return sample

def format_quarterly_date(date):
    # Create quarter format of date for visualisation purposes
    return f"{date.year}-Q{int((date.month-1) / 3) + 1}"

def evaluate_model_performance(sample, model, reporting=0):
    # Evaluation of model discriminatory power by calculating AUC values
    sample=sample.reset_index(drop=True)
    if reporting==1:
        sample=sample.drop(['reporting_date'], axis=1)
    if model==logreg:
        y_pred_proba = model.predict_proba(sample.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1))[:, 1]
    else:
        test_survival_probabilities = model.predict_surv_df(sample.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1).values.astype('float32')).reset_index(drop=True)
        duration_column = sample['duration']+12
        survival = pd.Series(test_survival_probabilities.T.lookup(sample.index, duration_column), index=sample.index)
        y_pred_proba=1-survival
    auc = roc_auc_score(sample['intodefault_12'], y_pred_proba)
    return auc, c_index

def integrated_calibration_index(y_true, y_prob, n_bins=10):
    # Sort the true labels and predicted probabilities based on the ascending order of probabilities
    sorted_indices = np.argsort(y_prob)
    y_true_sorted = y_true[sorted_indices]
    y_prob_sorted = y_prob[sorted_indices]

    # Calculate the bin size based on the length of y_true and the specified number of bins
    bin_size = len(y_true_sorted) // n_bins
    
    # Calculate the indices for dividing the sorted predictions into equal-sized bins
    bin_indices = np.arange(1, n_bins + 1) * bin_size

    # Calculate the proportion of positive labels in each bin
    bin_true_prob = np.array([np.mean(y_true_sorted[i-bin_size:i]) for i in bin_indices])

    # Calculate the mean predicted probability in each bin
    bin_pred_prob = np.array([np.mean(y_prob_sorted[i-bin_size:i]) for i in bin_indices])

    # Calculate the integrated calibration index
    ici = np.sum(np.abs(bin_true_prob - bin_pred_prob)) / n_bins

    return ici

def plot_calibration_curve(y, probs, title):
    # Brier Score
    brier_score = brier_score_loss(y, probs)
    # Integrated calibration index
    ici=integrated_calibration_index(y, probs)
    # Calculate proportion of positive labels and predicted probability in each of 10 bin (n_bins=10)
    prob_true, prob_pred = calibration_curve(y, probs, n_bins=10)
    # Plot calibration curve
    plt.figure(figsize=(6, 6)) 
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.plot(
        prob_pred,
        prob_true,
        marker=".",
        color="orange",
    )
    plt.title(f"{title}\nBrier score: {round(brier_score, 4)}\nICI: {round(ici, 4)}")
    plt.ylabel("Fraction of positives")
    plt.xlabel("Mean predicted value")
    return prob_true, prob_pred

def jeffreys_test(D, N, PD, confidence_level=0.95):
    # Calculate shape parameters for the beta distribution
    a = D + 0.5
    b = N - D + 0.5
    
    # Calculate the p-value using the cumulative distribution function (CDF)
    p_value = 1 - stats.beta.cdf(PD, a, b)
    
    # Calculate the Highest Density Interval (HDI) with the given confidence level
    beta_distribution = stats.beta(a, b)
    hdi = beta_distribution.interval(confidence_level)
    
    return p_value, hdi

def calculate_observed_loss(sample_table, reporting_date, time_horizon):
    # Calculation of observed loss for specified date and time horizon
    horizon_date = reporting_date + relativedelta(years=time_horizon)
    observed_loss = sample_table.loc[(sample_table['reporting_date'] == horizon_date) & (sample_table['default_adj'] == 1), 'Outstanding'].sum()
    return observed_loss

def calculate_corr(sample, sample_name, duration_ind=0):
    sample=sample.reset_index(drop=True)
    duration_column = sample['duration']+12
    
    # Logistic regression PD
    LR_PD = pd.Series(logreg.predict_proba(sample.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1))[:, 1])
    
    # Weibull AFT
    test_survival_probabilities = waft.predict_survival_function(sample, times=np.arange(1, 361))
    survival = pd.Series(test_survival_probabilities.T.lookup(sample.index, duration_column), index=sample.index)
    waft_PD=1-survival
    
    # Log-Normal AFT
    test_survival_probabilities = lognormaft.predict_survival_function(sample, times=np.arange(1, 361))
    survival = pd.Series(test_survival_probabilities.T.lookup(sample.index, duration_column), index=sample.index)
    lognormaft_PD=1-survival

    # Cox PH
    test_survival_probabilities = cph.predict_survival_function(sample, times=np.arange(1, 361))
    survival = pd.Series(test_survival_probabilities.T.lookup(sample.index, duration_column), index=sample.index)
    cph_PD=1-survival
    
    # DeepHit PD
    test_survival_probabilities = model.predict_surv_df(sample.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1).values.astype('float32'))
    survival = pd.Series(test_survival_probabilities.T.lookup(sample.index, duration_column), index=sample.index)
    DeepHit_PD=1-survival

    # Correlation matrix between PD results obtained from all models
    df_corr=pd.DataFrame({'Logistic regression': LR_PD, 'Weibull AFT':waft_PD, 'Log-Normal AFT': lognormaft_PD, 'Cox PH': cph_PD, 'DeepHit': DeepHit_PD})
    corr_matrix, p_value = stats.spearmanr(df_corr, axis=0)
    
    # Correlation plot between PD results obtained from all models
    plt.figure(figsize=(6, 6))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', xticklabels=df_corr.columns, yticklabels=df_corr.columns, fmt=".4f", cbar_kws={'label': 'Spearman Correlation Coefficient'})
    plt.title('Correlation matrix between PD predictions')
    plt.show()

    # p-values
    significant_mask=np.where(p_value<0.05,1,0)
    plt.figure(figsize=(6, 6))
    custom_pallette=["lightgreen","lightcoral"]
    heatmap=sns.heatmap(significant_mask, annot=p_value, cmap=custom_pallette, xticklabels=df_corr.columns, yticklabels=df_corr.columns, fmt=".2f", cbar=False)
    plt.title('P-values matrix between PD predictions')
    plt.show()
    
    # Define the pairs of x and y values and their corresponding titles
    pairs = [
        (LR_PD, DeepHit_PD, 'Logistic regression PD', 'DeepHit PD'),
        (lognormaft_PD, DeepHit_PD, 'Log-Normal AFT PD', 'DeepHit PD')
    ]

    # Iterate through the pairs and create scatterplots
    for x, y, x_label, y_label in pairs:
        plt.figure(figsize=(6, 6))
        plt.scatter(x, y)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        if duration_ind == 0:
            plt.title(f'Scatterplot between {y_label} and {x_label} predictions for {sample_name} sample')
        else:
            plt.title(f'Scatterplot between {y_label} and {x_label} predictions for {sample_name} sample for duration {duration}')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    
    return corr_matrix, p_value, DeepHit_PD, LR_PD, waft_PD, lognormaft_PD, cph_PD

## 1 Data upload and pre-processing

In [ ]:
df = pd.read_csv("sample_final_final.csv", delimiter=",")
df.drop('default', axis=1, inplace=True)
df.loc[(df['default_adj'] == 1) & (df['intodefault_12'] == 0), 'intodefault_12'] = 1
df.head()

In [ ]:
# Calculating missing values
missing_zero_values_table(df)

In [ ]:
# Replacing missing value with 0 for the given variables
df['rel_breach_pst_due_amt_o'] = df['rel_breach_pst_due_amt_o'].fillna(0)
df['absolute_breach_past_due_amnt_o'] = df['absolute_breach_past_due_amnt_o'].fillna(0)
df['diff_rel_breach_end_rep_date'] = df['diff_rel_breach_end_rep_date'].fillna(0)
df['diff_abs_breach_end_rep_date'] = df['diff_abs_breach_end_rep_date'].fillna(0)
df['diff_abs_breach_str_rep_date'] = df['diff_abs_breach_str_rep_date'].fillna(0)

df=df.rename(columns={"Average duration of existing installment credit lines. Calculated starting from the original durations of the individual credit lines.": "avg_duration_installment"})

# Datetime format for reporting_date
df['reporting_date'] = pd.to_datetime(df['reporting_date'])
df['min_rep_date'] = pd.to_datetime(df['min_rep_date'])

In [ ]:
# Dropping unnecessery variables for modelling purposes from df
data_dev = df.drop(['Arrear_amount_adj','first_def_date','mx_rep_Date','default_start_date','default_exit_date', 'Outstanding'], axis=1)

In [ ]:
# Split cid (facility id) into two groups
splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 11)
split = splitter.split(data_dev, groups=data_dev['cid'])
train_inds, test_inds = next(split)

sample_1 = data_dev.iloc[train_inds].reset_index(drop=True)
sample_2 = data_dev.iloc[test_inds].reset_index(drop=True)

In [ ]:
# Create training, OOS, and OOT samples
train = sample_1[sample_1['reporting_date'] < datetime(2019,7,1)].drop(['reporting_date', 'min_rep_date'], axis=1).reset_index(drop=True)
test = sample_2[sample_2['reporting_date'] < datetime(2019,7,1)].drop(['reporting_date', 'min_rep_date'], axis=1).reset_index(drop=True)
OOT_full = pd.concat([sample_2[sample_2['reporting_date'] >= datetime(2019, 7, 1)].drop(['reporting_date', 'min_rep_date'], axis=1),sample_1[sample_1['min_rep_date'] >= datetime(2019,7,1)].drop(['reporting_date', 'min_rep_date'], axis=1)],axis=0).reset_index(drop=True)

In [ ]:
# Pre-processing of samples
train=preprocessing(train)
test=preprocessing(test)
OOT_full=preprocessing(OOT_full)

In [ ]:
# Create OOT sample for comparing with logistic regression up to Dec 2020
merged_OOT = pd.merge(OOT_full, df[['cid', 'duration', 'reporting_date']], on=['cid', 'duration'], how='left')
OOT=merged_OOT[merged_OOT['reporting_date']<datetime(2021, 1, 1)].drop(['reporting_date'], axis=1).reset_index(drop=True)

In [ ]:
# Create X, y, t, id
X_train_on, Y_train_on, T_train_on, ID_train_on = train.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1), train['default_adj'], train['duration'], train['cid']
X_test_on, Y_test_on, T_test_on, ID_test_on = test.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1), test['default_adj'], test['duration'], test['cid']
X_test_OOT, Y_test_OOT, T_test_OOT, ID_test_OOT = OOT.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1), OOT['default_adj'], OOT['duration'], OOT['cid']
X_test_OOT_full, Y_test_OOT_full, T_test_OOT_full, ID_test_OOT_full = OOT_full.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1), OOT_full['default_adj'], OOT_full['duration'], OOT_full['cid']

In [ ]:
X_train_on.head()

## 2 Modelling & evaluation

### 2.1 Logistic Regression

In [ ]:
logreg = LogisticRegression()

# Train the model on the training data
logreg.fit(X_train_on, train['intodefault_12'])

In [ ]:
for sample_name, sample in [('train', train), ('test', test), ('OOT', OOT)]:
    y_pred_proba = logreg.predict_proba(sample.reset_index(drop=True).drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1))[:, 1]

    # Evaluate the model's performance
    auc = roc_auc_score(sample['intodefault_12'], y_pred_proba)
    print(f"AUC  for {sample_name} sample: {auc :.4f}")
    c_index = concordance_index(sample['duration'], -y_pred_proba, sample['intodefault_12'])
    print(f"C-index for {sample_name} sample: {c_index :.4f}")

In [ ]:
for sample_name, sample in [('train', train), ('test', test), ('OOT', OOT)]:
    df_sample = sample.copy()
    # Perform left join
    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'Arrear_amount_adj']], on=['cid', 'duration'], how='left')
    
    sub_segment_0 = merged_table[merged_table['Arrear_amount_adj'] <= 0]
    sub_segment_1 = merged_table[merged_table['Arrear_amount_adj'] > 0]
    
    y_pred_proba_0 = logreg.predict_proba(sub_segment_0.drop(['Arrear_amount_adj', 'intodefault_12', 'default_adj', 'duration', 'cid'], axis=1))[:, 1]
    y_pred_proba_1 = logreg.predict_proba(sub_segment_1.drop(['Arrear_amount_adj', 'intodefault_12', 'default_adj', 'duration', 'cid'], axis=1))[:, 1]

    # Evaluate the model's performance
    auc_0 = roc_auc_score(sub_segment_0['intodefault_12'], y_pred_proba_0)
    print(f"AUC  for segment arrears=0 {sample_name} sample: {auc_0 :.4f}")
    
    # Evaluate the model's performance
    auc_1 = roc_auc_score(sub_segment_1['intodefault_12'], y_pred_proba_1)
    print(f"AUC  for segment arrears=1 {sample_name} sample: {auc_1 :.4f}")

In [ ]:
out_of_bounds_months = []

for sample_name, sample in [('train', train), ('test', test), ('OOT', OOT)]:
    y_pred_proba = logreg.predict_proba(sample.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1))[:, 1]
    df_sample = sample.copy()
    df_sample['PD'] = y_pred_proba
    # Perform left join
    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date']], on=['cid', 'duration'], how='left')
    # Calculate sum of defaults and count of observations per reporting date
    summary = merged_table.groupby('reporting_date').agg({'intodefault_12': ['sum', 'count']})
    # Calculate default rate
    summary['Default Rate'] = summary[('intodefault_12', 'sum')] / summary[('intodefault_12', 'count')]
    # Calculate mean values per reporting date
    summary['PD'] = merged_table.groupby('reporting_date')['PD'].mean()
    summary['PD std'] = merged_table.groupby('reporting_date')['PD'].std()
    # Calculate 95% HDI of posterior distribution using binomial model with Jeffreys prior
    summary['jeffrey_p_value'], summary['HDI'] = zip(*summary.apply(lambda row: jeffreys_test(row['intodefault_12']['sum'], row['intodefault_12']['count'], row['PD']), axis=1))
    # Extract lower and upper bounds from HDI
    summary['LB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[0])
    summary['UB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[1])
    
    summary.sort_values('reporting_date', inplace=True)
    
    # Plotting the default rate and PD
    plt.figure(figsize=(10, 6))  # Adjust the figure size
    plt.plot(summary.index, summary['Default Rate'], label='Default Rate', color='blue', marker='o')
    plt.plot(summary.index, summary['PD'], label='PD', color='orange', marker='o')

    # Plotting 95% HDI of posterior distribution
    plt.fill_between(summary.index, summary['LB HDI 95%'], summary['UB HDI 95%'], linestyle='--', alpha=0.2, label='HDI 95%', color='blue')

    # Manually set x-axis tick labels to display quarterly reporting date
    quarterly_dates = pd.date_range(start=summary.index.min(), end=summary.index.max(), freq='Q')
    plt.xticks(quarterly_dates, [format_quarterly_date(date) for date in quarterly_dates], rotation=45)

    # Adding labels and title
    plt.xlabel('Reporting Date')
    plt.ylabel('Rate')
    plt.title(f'Into default Rate vs. PD over Time for sample {sample_name}')
    plt.legend(loc='upper left')

    # Display gridlines for better readability
    plt.grid(True)

    # Display the graph
    plt.tight_layout()  # Ensure the labels are not cut off
    plt.show()
    # Count the number of months the default rate is out of confidence bounds
    out_of_bounds = np.where(
        (summary['PD'] < summary['LB HDI 95%']) | (summary['PD'] > summary['UB HDI 95%'])
    )[0]
    out_of_bounds_months.append(len(out_of_bounds))

    prob_true, prob_pred=plot_calibration_curve(merged_table['intodefault_12'], merged_table['PD'], f'Calibration curve sample {sample_name}')
    plt.tight_layout()
    plt.show()

# Print the number of months the default rate is out of confidence bounds for each sample
for i, sample_name in enumerate(['train', 'test', 'OOT']):
    print(f"Number of months out of bounds for {sample_name}: {out_of_bounds_months[i]}")

In [ ]:
for sample_name, sample in [('OOT', OOT_full)]:
    # As an input to evaluate logistic regression results OOT_full is used as it contains observed loss information for 2021 data
    y_pred_proba = logreg.predict_proba(sample.drop(['intodefault_12', 'default_adj', 'duration', 'cid'], axis=1))[:, 1]
    df_sample = sample.copy()
    df_sample['PD'] = y_pred_proba
    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Outstanding']], on=['cid', 'duration'], how='left')

    merged_table['EL'] = merged_table['PD'] * merged_table['Outstanding']/1000000
    merged_table['OL'] = merged_table.loc[merged_table['default_adj'] == 1, 'Outstanding']/1000000
    merged_table['OL'] = merged_table['OL'].fillna(0)

    expected_loss = merged_table.groupby('reporting_date')['EL'].sum().rename('Expected Loss').reset_index()
    observed_loss = merged_table.groupby('reporting_date')['OL'].sum().rename('Observed Loss').reset_index()

    observed_loss_comparison = observed_loss.copy()

    # Add a new column for the corresponding observed loss one year after
    observed_loss_comparison['Observed Loss (One Year After)'] = None

    # Calculate the corresponding observed loss one year after for each reporting date
    for i, row in observed_loss.iterrows():
        current_date = row['reporting_date']
        current_month = current_date.month
        current_year = current_date.year

        # Check for February 28th in a leap year
        if current_month == 2 and current_date.day == 28 and current_year % 4 == 3:
            # Calculate the corresponding date one year after, considering leap years
            one_year_after = current_date + relativedelta(years=1, day=29)
        else:
            # Calculate the corresponding date one year after
            one_year_after = current_date + relativedelta(years=1)

        one_year_after_loss = observed_loss[observed_loss['reporting_date'] == one_year_after]['Observed Loss']

        if not one_year_after_loss.empty:
            observed_loss_comparison.at[i, 'One-Year Observed Loss'] = one_year_after_loss.iloc[0]

    observed_loss_comparison = observed_loss_comparison[observed_loss_comparison['reporting_date'] < datetime(2021, 1, 1)]
    expected_loss = expected_loss[expected_loss['reporting_date'] < datetime(2021, 1, 1)]
    merged_table = merged_table[merged_table['reporting_date'] < datetime(2021, 1, 1)]

    expected_loss.set_index('reporting_date', inplace=True)
    observed_loss_comparison.set_index('reporting_date', inplace=True)

    summary = pd.concat([expected_loss, observed_loss_comparison['One-Year Observed Loss']], axis=1)
    summary['Difference'] = summary['Expected Loss'] - summary['One-Year Observed Loss']

    # Plotting the data
    plt.figure(figsize=(10, 6))  # Adjust the figure size
    plt.plot(observed_loss_comparison.index, observed_loss_comparison['One-Year Observed Loss'], label='Observed Loss', marker='o')
    plt.plot(expected_loss.index, expected_loss['Expected Loss'], label='Expected Loss', marker='o')

    quarterly_dates = pd.date_range(start=summary.index.min(), end=summary.index.max(), freq='Q')
    plt.xticks(quarterly_dates, [format_quarterly_date(date) for date in quarterly_dates], rotation=45)

    plt.xlabel('Reporting Date')
    plt.ylabel('Loss (in millions)')
    plt.title(f'Expected Loss vs. Observed Loss over Time for sample {sample_name}')
    plt.legend(loc='upper left')

    plt.grid(True)

    plt.tight_layout()
    plt.show()

### 2.2 Survival analysis

**WeibullAFTFitter**

In [ ]:
# Weibull AFT in lifelines library need duration time starting from 1 (not 0)
T_train_on_adj=T_train_on+1
T_test_on_adj=T_test_on+1
T_test_OOT_adj=T_test_OOT+1
timeline = np.arange(1, 361)

In [ ]:
waft = WeibullAFTFitter(penalizer=10)

waft.fit(pd.concat([pd.DataFrame(X_train_on).reset_index(drop=True), T_train_on_adj.reset_index(drop=True), Y_train_on.reset_index(drop=True)], axis=1), 'duration', event_col='default_adj')

In [ ]:
#Evaluation
out_of_bounds_months = []
auc_list = []
c_index_list = []

for sample_name, sample in [('train', train), ('test', test), ('OOT', OOT)]:
    reset_sample = sample.reset_index(drop=True)
    test_survival_probabilities = waft.predict_survival_function(reset_sample, times=np.arange(1, 361)).reset_index(drop=True)
    
    duration_column = reset_sample['duration']+12
    survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)
    
    df_sample=sample.reset_index(drop=True).copy()
    df_sample['PD']=1-survival

    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Arrear_amount_adj']], on=['cid', 'duration'], how='left')

    summary = merged_table.groupby('reporting_date').agg({'intodefault_12': ['sum', 'count']})
    summary['Default Rate'] = summary[('intodefault_12', 'sum')] / summary[('intodefault_12', 'count')]
    summary['PD'] = merged_table.groupby('reporting_date')['PD'].mean()
    summary['PD std'] = merged_table.groupby('reporting_date')['PD'].std()
    # Evaluate predictive ability
    summary['jeffrey_p_value'], summary['HDI'] = zip(*summary.apply(lambda row: jeffreys_test(row['intodefault_12']['sum'], row['intodefault_12']['count'], row['PD']), axis=1))
    summary['LB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[0])
    summary['UB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[1])
    
    summary.sort_values('reporting_date', inplace=True)
    
    plt.figure(figsize=(10, 6))
    plt.plot(summary.index, summary['Default Rate'], label='Default Rate', color='blue', marker='o')
    plt.plot(summary.index, summary['PD'], label='PD', color='orange', marker='o')
    plt.fill_between(summary.index, summary['LB HDI 95%'], summary['UB HDI 95%'], linestyle='--', alpha=0.2, color='blue', label="HDI")

    quarterly_dates = pd.date_range(start=summary.index.min(), end=summary.index.max(), freq='Q')
    plt.xticks(quarterly_dates, [format_quarterly_date(date) for date in quarterly_dates], rotation=45)

    plt.xlabel('Reporting date')
    plt.ylabel('Rate')
    plt.title(f'Into default Rate vs. PD over Time for sample {sample_name}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    out_of_bounds = np.where(
        (summary['PD'] < summary['LB HDI 95%']) | (summary['PD'] > summary['UB HDI 95%'])
    )[0]
    out_of_bounds_months.append(len(out_of_bounds))
    
    # Evaluate the discriminatory power
    auc = roc_auc_score(merged_table['intodefault_12'], merged_table['PD'])
    auc_list.append(auc)
    
    c_index = concordance_index(merged_table['duration'], -merged_table['PD'], merged_table['intodefault_12'])
    c_index_list.append(c_index)
                                
    plot_calibration_curve(merged_table['intodefault_12'], merged_table['PD'], f'Calibration curve sample {sample_name}')
    plt.show()

# Print the number of months the default rate is out of confidence bounds for each sample
for i, sample_name in enumerate(['train', 'test', 'OOT']):
    print(f"Number of months default rate is out of bounds for {sample_name}: {out_of_bounds_months[i]}")
    print(f"AUC for {sample_name}: {auc_list[i]:.4f}")
    print(f"C-index for {sample_name}: {c_index_list[i]:.4f}")

In [ ]:
# Create an empty DataFrame to store the results
merged_test_oot = pd.concat([test,OOT_full])
result_table = pd.DataFrame()

# Iterate over different reporting dates (December only)
for reporting_date in pd.date_range(start='2013-12-01', end='2021-12-31', freq='12M'):
    # Calculate the corresponding date for each time horizon of 0 to 8 years ahead
    time_horizons = list(np.arange(0,9))
    corresponding_dates = [reporting_date + relativedelta(years=t) for t in time_horizons]
    corresponding_dates = [date for date in corresponding_dates if date < pd.Timestamp('2022-01-01')]
    time_horizons = np.arange(len(corresponding_dates))
    # Initialize lists to store the results for the current reporting date
    reporting_dates_list = [reporting_date] * len(time_horizons)
    time_horizons_list = time_horizons.copy()
    corresponding_dates_list = corresponding_dates
    expected_losses = []
    observed_losses = []

    for sample_name, sample in [('Merged', merged_test_oot)]:
        reset_sample = sample.reset_index(drop=True)

        test_survival_probabilities = waft.predict_survival_function(reset_sample, times=np.arange(1, 361)).reset_index(drop=True)

        for years_ahead in time_horizons:
            duration_column = reset_sample['duration'] + 12 * years_ahead
            survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)

            df_sample = sample.reset_index(drop=True).copy()
            df_sample['PD'] = 1 - survival

            merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Outstanding']], on=['cid', 'duration'], how='left')

            merged_table['EL'] = merged_table['PD'] * merged_table['Outstanding']/1000000

            # Calculate Expected Loss for the current time horizon
            expected_loss = merged_table.groupby('reporting_date')['EL'].sum().rename('Expected Loss').reset_index()
            expected_loss = expected_loss[expected_loss['reporting_date'] == reporting_date]

            # Calculate Observed Loss for the current time horizon
            observed_loss = calculate_observed_loss(merged_table, reporting_date, years_ahead)

            # Append the results to the lists
            expected_losses.extend(expected_loss['Expected Loss'])
            observed_losses.append(observed_loss/1000000)

    # Append the results for the current reporting date to the result table
    data = {
        'Reporting Date': reporting_dates_list,
        'Time Horizon': time_horizons_list,
        'Corresponding Date': corresponding_dates_list,
        'Expected Loss': expected_losses,
        'Observed Loss': observed_losses
    }
    result_table = result_table.append(pd.DataFrame(data), ignore_index=True)

In [ ]:
result_table

**LogNormalAFTFitter**

In [ ]:
lognormaft = LogNormalAFTFitter(penalizer=10)

lognormaft.fit(pd.concat([pd.DataFrame(X_train_on).reset_index(drop=True), T_train_on_adj.reset_index(drop=True), Y_train_on.reset_index(drop=True)], axis=1), 'duration', event_col='default_adj')

In [ ]:
out_of_bounds_months = []
auc_list = []
c_index_list = []

for sample_name, sample in [('train', train), ('test', test), ('OOT', OOT)]:
    reset_sample = sample.reset_index(drop=True)
    test_survival_probabilities = lognormaft.predict_survival_function(reset_sample, times=np.arange(1, 361)).reset_index(drop=True)
    
    duration_column = reset_sample['duration']+12
    survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)
    
    df_sample=sample.reset_index(drop=True).copy()
    df_sample['PD']=1-survival

    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Arrear_amount_adj']], on=['cid', 'duration'], how='left')

    summary = merged_table.groupby('reporting_date').agg({'intodefault_12': ['sum', 'count']})
    summary['Default Rate'] = summary[('intodefault_12', 'sum')] / summary[('intodefault_12', 'count')]
    summary['PD'] = merged_table.groupby('reporting_date')['PD'].mean()
    summary['PD std'] = merged_table.groupby('reporting_date')['PD'].std()
    # Evaluate predictive ability
    summary['jeffrey_p_value'], summary['HDI'] = zip(*summary.apply(lambda row: jeffreys_test(row['intodefault_12']['sum'], row['intodefault_12']['count'], row['PD']), axis=1))
    summary['LB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[0])
    summary['UB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[1])
    
    summary.sort_values('reporting_date', inplace=True)
    
    plt.figure(figsize=(10, 6))
    plt.plot(summary.index, summary['Default Rate'], label='Default Rate', color='blue', marker='o')
    plt.plot(summary.index, summary['PD'], label='PD', color='orange', marker='o')
    plt.fill_between(summary.index, summary['LB HDI 95%'], summary['UB HDI 95%'], linestyle='--', alpha=0.2, color='blue', label="HDI")

    quarterly_dates = pd.date_range(start=summary.index.min(), end=summary.index.max(), freq='Q')
    plt.xticks(quarterly_dates, [format_quarterly_date(date) for date in quarterly_dates], rotation=45)

    plt.xlabel('Reporting date')
    plt.ylabel('Rate')
    plt.title(f'Into default Rate vs. PD over Time for sample {sample_name}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    out_of_bounds = np.where(
        (summary['PD'] < summary['LB HDI 95%']) | (summary['PD'] > summary['UB HDI 95%'])
    )[0]
    out_of_bounds_months.append(len(out_of_bounds))
    
    # Evaluate the discriminatory power
    auc = roc_auc_score(merged_table['intodefault_12'], merged_table['PD'])
    auc_list.append(auc)
    
    c_index = concordance_index(merged_table['duration'], -merged_table['PD'], merged_table['intodefault_12'])
    c_index_list.append(c_index)
                                
    plot_calibration_curve(merged_table['intodefault_12'], merged_table['PD'], f'Calibration curve sample {sample_name}')
    plt.show()

for i, sample_name in enumerate(['train', 'test', 'OOT']):
    print(f"Number of months default rate is out of bounds for {sample_name}: {out_of_bounds_months[i]}")
    print(f"AUC for {sample_name}: {auc_list[i]:.4f}")
    print(f"C-index for {sample_name}: {c_index_list[i]:.4f}")

In [ ]:
result_table = pd.DataFrame()

# Iterate over different reporting dates (December only)
for reporting_date in pd.date_range(start='2013-12-01', end='2021-12-31', freq='12M'):
    time_horizons = list(np.arange(0,9))
    corresponding_dates = [reporting_date + relativedelta(years=t) for t in time_horizons]
    corresponding_dates = [date for date in corresponding_dates if date < pd.Timestamp('2022-01-01')]
    time_horizons = np.arange(len(corresponding_dates))
    reporting_dates_list = [reporting_date] * len(time_horizons)
    time_horizons_list = time_horizons.copy()
    corresponding_dates_list = corresponding_dates
    expected_losses = []
    observed_losses = []

    for sample_name, sample in [('Merged', merged_test_oot)]:
        reset_sample = sample.reset_index(drop=True)

        test_survival_probabilities = lognormaft.predict_survival_function(reset_sample, times=np.arange(1, 361)).reset_index(drop=True)

        for years_ahead in time_horizons:
            duration_column = reset_sample['duration'] + 12 * years_ahead
            survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)

            df_sample = sample.reset_index(drop=True).copy()
            df_sample['PD'] = 1 - survival
            merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Outstanding']], on=['cid', 'duration'], how='left')
            merged_table['EL'] = merged_table['PD'] * merged_table['Outstanding']/1000000

            expected_loss = merged_table.groupby('reporting_date')['EL'].sum().rename('Expected Loss').reset_index()
            expected_loss = expected_loss[expected_loss['reporting_date'] == reporting_date]
            observed_loss = calculate_observed_loss(merged_table, reporting_date, years_ahead)
            expected_losses.extend(expected_loss['Expected Loss']/1000000)
            observed_losses.append(observed_loss)

    data = {
        'Reporting Date': reporting_dates_list,
        'Time Horizon': time_horizons_list,
        'Corresponding Date': corresponding_dates_list,
        'Expected Loss': expected_losses,
        'Observed Loss': observed_losses
    }
    result_table = result_table.append(pd.DataFrame(data), ignore_index=True)

In [ ]:
result_table

**CoxPHFitter**

In [ ]:
cph = CoxPHFitter(penalizer=10)

cph.fit(pd.concat([pd.DataFrame(X_train_on), T_train_on.reset_index(drop=True), Y_train_on.reset_index(drop=True)], axis=1), duration_col='duration', event_col='default_adj')

cph.print_summary()

In [ ]:
out_of_bounds_months = []
auc_list = []
c_index_list = []

for sample_name, sample in [('train', train), ('test', test), ('OOT', OOT)]:
    reset_sample = sample.reset_index(drop=True)
    test_survival_probabilities = cph.predict_survival_function(reset_sample, times=np.arange(1, 361)).reset_index(drop=True)
    
    duration_column = reset_sample['duration']+12
    survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)
    
    df_sample=sample.reset_index(drop=True).copy()
    df_sample['PD']=1-survival

    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Arrear_amount_adj']], on=['cid', 'duration'], how='left')

    summary = merged_table.groupby('reporting_date').agg({'intodefault_12': ['sum', 'count']})
    summary['Default Rate'] = summary[('intodefault_12', 'sum')] / summary[('intodefault_12', 'count')]
    summary['PD'] = merged_table.groupby('reporting_date')['PD'].mean()
    summary['PD std'] = merged_table.groupby('reporting_date')['PD'].std()
    # Evaluate predictive ability
    summary['jeffrey_p_value'], summary['HDI'] = zip(*summary.apply(lambda row: jeffreys_test(row['intodefault_12']['sum'], row['intodefault_12']['count'], row['PD']), axis=1))
    summary['LB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[0])
    summary['UB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[1])
    
    summary.sort_values('reporting_date', inplace=True)
    
    plt.figure(figsize=(10, 6))
    plt.plot(summary.index, summary['Default Rate'], label='Default Rate', color='blue', marker='o')
    plt.plot(summary.index, summary['PD'], label='PD', color='orange', marker='o')
    plt.fill_between(summary.index, summary['LB HDI 95%'], summary['UB HDI 95%'], linestyle='--', alpha=0.2, color='blue', label="HDI")

    quarterly_dates = pd.date_range(start=summary.index.min(), end=summary.index.max(), freq='Q')
    plt.xticks(quarterly_dates, [format_quarterly_date(date) for date in quarterly_dates], rotation=45)
    
    plt.xlabel('Reporting date')
    plt.ylabel('Rate')
    plt.title(f'Into default Rate vs. PD over Time for sample {sample_name}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    out_of_bounds = np.where(
        (summary['PD'] < summary['LB HDI 95%']) | (summary['PD'] > summary['UB HDI 95%'])
    )[0]
    out_of_bounds_months.append(len(out_of_bounds))
    
    # Evaluate the discriminatory power
    auc = roc_auc_score(merged_table['intodefault_12'], merged_table['PD'])
    auc_list.append(auc)
    
    c_index = concordance_index(merged_table['duration'], -merged_table['PD'], merged_table['intodefault_12'])
    c_index_list.append(c_index)
                                
    plot_calibration_curve(merged_table['intodefault_12'], merged_table['PD'], f'Calibration curve sample {sample_name}')
    plt.show()

for i, sample_name in enumerate(['train', 'test', 'OOT']):
    print(f"Number of months default rate is out of bounds for {sample_name}: {out_of_bounds_months[i]}")
    print(f"AUC for {sample_name}: {auc_list[i]:.4f}")
    print(f"C-index for {sample_name}: {c_index_list[i]:.4f}")

In [ ]:
result_table = pd.DataFrame()

# Iterate over different reporting dates (December only)
for reporting_date in pd.date_range(start='2013-12-01', end='2021-12-31', freq='12M'):
    time_horizons = list(np.arange(0,9))
    corresponding_dates = [reporting_date + relativedelta(years=t) for t in time_horizons]
    corresponding_dates = [date for date in corresponding_dates if date < pd.Timestamp('2022-01-01')]
    time_horizons = np.arange(len(corresponding_dates))
    reporting_dates_list = [reporting_date] * len(time_horizons)
    time_horizons_list = time_horizons.copy()
    corresponding_dates_list = corresponding_dates
    expected_losses = []
    observed_losses = []

    for sample_name, sample in [('Merged', merged_test_oot)]:
        reset_sample = sample.reset_index(drop=True)

        test_survival_probabilities = cph.predict_survival_function(reset_sample, times=np.arange(1, 361)).reset_index(drop=True)

        for years_ahead in time_horizons:
            duration_column = reset_sample['duration'] + 12 * years_ahead
            survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)

            df_sample = sample.reset_index(drop=True).copy()
            df_sample['PD'] = 1 - survival

            merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Outstanding']], on=['cid', 'duration'], how='left')
            merged_table['EL'] = merged_table['PD'] * merged_table['Outstanding']/1000000

            expected_loss = merged_table.groupby('reporting_date')['EL'].sum().rename('Expected Loss').reset_index()
            expected_loss = expected_loss[expected_loss['reporting_date'] == reporting_date]
            observed_loss = calculate_observed_loss(merged_table, reporting_date, years_ahead)
            expected_losses.extend(expected_loss['Expected Loss'])
            observed_losses.append(observed_loss/1000000)

    data = {
        'Reporting Date': reporting_dates_list,
        'Time Horizon': time_horizons_list,
        'Corresponding Date': corresponding_dates_list,
        'Expected Loss': expected_losses,
        'Observed Loss': observed_losses
    }
    result_table = result_table.append(pd.DataFrame(data), ignore_index=True)

In [ ]:
result_table

**DeepHit**

In [ ]:
# Parameters of model to be used during training
duration_index = np.arange(1, 361)
num_durations = len(duration_index)
labtrans = DeepHitSingle.label_transform(num_durations)
get_target = lambda df: (df['duration'].values, df['default_adj'].values)
y_train = labtrans.fit_transform(*get_target(train))
y_val = labtrans.transform(*get_target(test))
in_features = X_train_on.shape[1]
out_features = labtrans.out_features

In [ ]:
# Hypertuning
# Create space of hyperparameters values to serach
space = {
    'alpha': hp.uniform('alpha', 0, 0.5),
    'sigma': hp.uniform('sigma', 0, 5),
    'num_nodes': hp.quniform('num_nodes', 100, 5000, 100),
    'num_layers': hp.quniform('num_layers', 1, 4, 1),  # Fix the list
    'batch_norm': hp.choice('batch_norm', [True, False]),
    'dropout_rate': hp.uniform('dropout_rate', 0, 0.5),  # Fix the name
    'batch_size': hp.quniform('batch_size', 100, 5000, 100)
}

# Define objective function
def objective(params):
    # Set seed
    np.random.seed(123)
    _ = torch.manual_seed(123)
    
    # Cast to an integer
    params['num_nodes'] = int(params['num_nodes'])
    params['num_layers'] = int(params['num_layers'])
    params['batch_size'] = int(params['batch_size'])
    
    # Crate net
    net = tt.practical.MLPVanilla(in_features, [params['num_nodes']] * params['num_layers'], out_features, params['batch_norm'], params['dropout_rate'])
    model = DeepHitSingle(net, tt.optim.Adam, alpha=params['alpha'], sigma=params['sigma'], duration_index=np.arange(1, 361))
    model.optimizer.set_lr(0.0001)
    epochs = 5
    callbacks = [tt.callbacks.EarlyStopping()]
    log = model.fit(X_train_on.values.astype('float32'), (T_train_on.values, Y_train_on.values), params['batch_size'], epochs, callbacks, verbose=0, val_data=(X_test_on.values.astype('float32'), (T_test_on.values, Y_test_on.values)))
    
    train_survival_probabilities = model.predict_surv_df(X_test_OOT_full.values.astype('float32')).reset_index(drop=True)  # Reset index here
    duration_column = OOT_full['duration'] + 12
    survival = pd.Series(train_survival_probabilities.T.lookup(OOT_full.reset_index(drop=True).index, duration_column), index=OOT_full.reset_index(drop=True).index)

    df_sample = OOT_full.reset_index(drop=True).copy()
    df_sample['PD'] = 1 - survival
    
    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Outstanding']], on=['cid', 'duration'], how='left')

    auc = roc_auc_score(merged_table['intodefault_12'], merged_table['PD'])
    # Since the minimize function is used further, -auc is returned
    return -auc

# Run hyperparameter search
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

# Print best hyperparameters
print('Best hyperparameters:', best)

In [ ]:
#training of DeepHit model
np.random.seed(123)
_ = torch.manual_seed(123)

#net parameters
num_nodes = 700
num_layers = 1
batch_norm = False
dropout = 0.25
net = tt.practical.MLPVanilla(in_features, [num_nodes]*num_layers, out_features, batch_norm, dropout)

model = DeepHitSingle(net, tt.optim.Adam, alpha=0.4, sigma=2.5, duration_index=np.arange(1,361))
model.optimizer.set_lr(0.0001)
epochs = 10
callbacks = [tt.callbacks.EarlyStopping()]
log = model.fit(X_train_on.values.astype('float32'), (T_train_on.values,Y_train_on.values), 2900, epochs, callbacks, verbose=1, val_data=(X_test_on.values.astype('float32'), (T_test_on.values,Y_test_on.values)))

In [ ]:
out_of_bounds_months = []
auc_list = []
auc_0 = []

for sample_name, sample, X in [('test', test, X_test_on), ('OOT', OOT, X_test_OOT)]:
    reset_sample = sample.reset_index(drop=True)

    test_survival_probabilities = model.predict_surv_df(X.values.astype('float32')).reset_index(drop=True)
    
    duration_column = reset_sample['duration']+12
    survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)
    
    df_sample=sample.reset_index(drop=True).copy()
    df_sample['PD']=1-survival
    # Perform left join to add reporting date and arrears information
    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Arrear_amount_adj']], on=['cid', 'duration'], how='left')

    summary = merged_table.groupby('reporting_date').agg({'intodefault_12': ['sum', 'count']})
    summary['Default Rate'] = summary[('intodefault_12', 'sum')] / summary[('intodefault_12', 'count')]
    summary['PD'] = merged_table.groupby('reporting_date')['PD'].mean()
    summary['PD std'] = merged_table.groupby('reporting_date')['PD'].std()
    # Evaluate predictive ability
    summary['jeffrey_p_value'], summary['HDI'] = zip(*summary.apply(lambda row: jeffreys_test(row['intodefault_12']['sum'], row['intodefault_12']['count'], row['PD']), axis=1))
    summary['LB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[0])
    summary['UB HDI 95%'] = summary['HDI'].apply(lambda hdi: hdi[1])
    
    summary.sort_values('reporting_date', inplace=True)
    
    plt.figure(figsize=(10, 6))
    plt.plot(summary.index, summary['Default Rate'], label='Default Rate', color='blue', marker='o')
    plt.plot(summary.index, summary['PD'], label='PD', color='orange', marker='o')
    plt.fill_between(summary.index, summary['LB HDI 95%'], summary['UB HDI 95%'], linestyle='--', alpha=0.2, color='blue')

    quarterly_dates = pd.date_range(start=summary.index.min(), end=summary.index.max(), freq='Q')
    plt.xticks(quarterly_dates, [format_quarterly_date(date) for date in quarterly_dates], rotation=45)

    plt.xlabel('Reporting date')
    plt.ylabel('Rate')
    plt.title(f'Into default Rate vs. PD over Time for sample {sample_name}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    out_of_bounds = np.where(
        (summary['PD'] < summary['LB HDI 95%']) | (summary['PD'] > summary['UB HDI 95%'])
    )[0]
    out_of_bounds_months.append(len(out_of_bounds))
    
    # Evaluate the discriminatory power at the portfolio level
    auc = roc_auc_score(merged_table['intodefault_12'], merged_table['PD'])
    auc_list.append(auc)
    
    # Evaluate the discriminatory power at the material sub-ranges level
    sub_segment_0 = merged_table[merged_table['Arrear_amount_adj'] == 0]
    sub_segment_1 = merged_table[merged_table['Arrear_amount_adj'] > 0]
    
    auc_0 = roc_auc_score(sub_segment_0['intodefault_12'], sub_segment_0['PD'])
    print(f"AUC  for segment arrears=0 {sample_name} sample: {auc_0 :.4f}")
    auc_1 = roc_auc_score(sub_segment_1['intodefault_12'], sub_segment_1['PD'])
    print(f"AUC  for segment arrears=1 {sample_name} sample: {auc_1 :.4f}")
    
    plot_calibration_curve(merged_table['intodefault_12'], merged_table['PD'], f'Calibration curve sample {sample_name}')
    plt.tight_layout()
    plt.show()

for i, sample_name in enumerate(['test', 'OOT']):
    print(f"Number of months default rate is out of bounds for {sample_name}: {out_of_bounds_months[i]}")
    print(f"AUC for {sample_name}: {auc_list[i]:.4f}")

In [ ]:
# AUC values over time for the first set of AUC values
merged_test_oot_notfull = pd.concat([test,OOT])
evaluation_results_1 = []
evaluation_results_2 = []
for samplename, sample in [('OOS and OOT combined', merged_test_oot_notfull)]:
    auc_values_1 = []
    auc_values_2 = []
    df_sample = sample.copy()
    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date']], on=['cid', 'duration'], how='left')
    
    reporting_dates_to_evaluate = sorted(merged_table['reporting_date'].unique())

    for reporting_date in reporting_dates_to_evaluate:
        sample_time = merged_table[merged_table['reporting_date'] == reporting_date]
        
        if len(sample_time['intodefault_12'].unique()) == 1:
            auc_values_1.append(None)
            auc_values_2.append(None)
        else:
            auc, _ = evaluate_model_performance(sample_time, logreg, reporting=1)
            auc_values_1.append(auc)
            
            auc, _ = evaluate_model_performance(sample_time, model, reporting=1)
            auc_values_2.append(auc)

    evaluation_results_1.append({
        'dates': reporting_dates_to_evaluate,
        'Sample': samplename,
        'AUC Values': auc_values_1
    })
    evaluation_results_2.append({
        'dates': reporting_dates_to_evaluate,
        'Sample': samplename,
        'AUC Values': auc_values_2
    })

# Combine the two sets of AUC values into one plot
plt.figure(figsize=(10, 6))
for evaluation in evaluation_results_1:
    plt.plot(evaluation['dates'], evaluation['AUC Values'], label=f'Logistic regression AUC', marker='o')
for evaluation in evaluation_results_2:
    plt.plot(evaluation['dates'], evaluation['AUC Values'], label=f'DeepHit AUC', marker='o') 

oot_threshold_date=datetime(2019,7,31)

plt.axvspan(reporting_dates_to_evaluate[0], oot_threshold_date, facecolor='lightgray', alpha=0.5)
plt.axvspan(oot_threshold_date, reporting_dates_to_evaluate[-1], facecolor='lightcoral', alpha=0.5)

# Add a horizontal line for July 2019
plt.axvline(x=oot_threshold_date, color='gray', linestyle='--', linewidth=2)

# Add text to indicate OOT and OOS
plt.text(pd.to_datetime('2019-08-31'), 0.86, 'OOT', fontsize=12, color='black')
plt.text(pd.to_datetime('2019-03-01'), 0.86, 'OOS', fontsize=12, color='black')

plt.xlabel('Reporting Date')
plt.ylabel('AUC')
plt.title('AUC for Different Reporting Dates over Time')
plt.xlim(reporting_dates_to_evaluate[0],reporting_dates_to_evaluate[-1])
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Define a list of durations to evaluate
durations_to_evaluate = [12, 24, 36, 48, 60]

# AUC values over time for the second set of AUC values
evaluation_results_1 = []
evaluation_results_2 = []
for samplename, sample in [('OOT', OOT)]:
    auc_values_1 = []
    auc_values_2 = []
    for duration in durations_to_evaluate:
        # Filter data based on the duration
        sample_time = sample[sample['duration'] == duration]
        
        # Evaluate the model's performance for the specific duration at the specific reporting_date
        auc, _ = evaluate_model_performance(sample_time, logreg)
        # Append the AUC value to the list for the specific duration 
        auc_values_1.append(auc)
        
        auc, _ = evaluate_model_performance(sample_time, model)
        auc_values_2.append(auc)

    # Append the results to the evaluation_results list
    evaluation_results_1.append({
        'duration': duration,
        'Sample': sample_name,
        'AUC Values': auc_values_1
    })
    evaluation_results_2.append({
        'duration': duration,
        'Sample': sample_name,
        'AUC Values': auc_values_2
    })
    
# Combine the two sets of AUC values into one plot
plt.figure(figsize=(10, 6))

# Plotting the AUC for different durations over time
for evaluation in evaluation_results_1:
    plt.plot(durations_to_evaluate, evaluation['AUC Values'], label=f'Logistic regression AUC', marker='o')
for evaluation in evaluation_results_2:
    plt.plot(durations_to_evaluate, evaluation['AUC Values'], label=f'DeepHit AUC', marker='o')

plt.xlabel('duration')
plt.xticks([12, 24, 36, 48, 60])
plt.ylabel('AUC')
plt.title(f'AUC for Different Durations over Time for sample {evaluation["Sample"]}')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ECL vs OL over one-year horizon on OOT evaluation
for sample_name, sample in [('OOT', OOT_full)]:
    reset_sample = sample.reset_index(drop=True)

    test_survival_probabilities = model.predict_surv_df(X_test_OOT_full.values.astype('float32')).reset_index(drop=True)
    
    duration_column = reset_sample['duration']+12
    survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)
    
    df_sample=sample.reset_index(drop=True).copy()
    df_sample['PD']=1-survival
    
    merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Outstanding']], on=['cid', 'duration'], how='left')

    merged_table['EL'] = merged_table['PD'] * (merged_table['Outstanding']-12*merged_table['Outstanding']/(360-reset_sample['duration']))/1000000
    merged_table['OL'] = merged_table.loc[merged_table['default_adj'] == 1, 'Outstanding']/1000000
    merged_table['OL'] = merged_table['OL'].fillna(0)

    expected_loss = merged_table.groupby('reporting_date')['EL'].sum().rename('Expected Loss').reset_index()
    observed_loss = merged_table.groupby('reporting_date')['OL'].sum().rename('Observed Loss').reset_index()

    observed_loss_comparison = observed_loss.copy()
    observed_loss_comparison['Observed Loss (One Year After)'] = None

    for i, row in observed_loss.iterrows():
        current_date = row['reporting_date']
        current_month = current_date.month
        current_year = current_date.year

        if current_month == 2 and current_date.day == 28 and current_year % 4 == 3:
            one_year_after = current_date + relativedelta(years=1, day=29)
        else:
            one_year_after = current_date + relativedelta(years=1)

        one_year_after_loss = observed_loss[observed_loss['reporting_date'] == one_year_after]['Observed Loss']

        if not one_year_after_loss.empty:
            observed_loss_comparison.at[i, 'One-Year Observed Loss'] = one_year_after_loss.iloc[0]

    observed_loss_comparison = observed_loss_comparison[observed_loss_comparison['reporting_date'] < datetime(2021, 1, 1)]
    expected_loss = expected_loss[expected_loss['reporting_date'] < datetime(2021, 1, 1)]
    merged_table = merged_table[merged_table['reporting_date'] < datetime(2021, 1, 1)]

    # Combine expected_loss, observed_loss_comparison, and the difference between the two
    summary = pd.concat([expected_loss, observed_loss_comparison['One-Year Observed Loss']], axis=1)
    summary['Difference'] = summary['Expected Loss'] - summary['One-Year Observed Loss']
    
    plt.figure(figsize=(10, 6))
    plt.plot(observed_loss_comparison['reporting_date'], observed_loss_comparison['One-Year Observed Loss'], label='Observed Loss', marker='o')
    plt.plot(expected_loss['reporting_date'], expected_loss['Expected Loss'], label='Expected Loss', marker='o')

    quarterly_dates = pd.date_range(start=expected_loss['reporting_date'].min(), end=expected_loss['reporting_date'].max(), freq='Q')
    plt.xticks(quarterly_dates, [format_quarterly_date(date) for date in quarterly_dates], rotation=45)

    plt.xlabel('Reporting date')
    plt.ylabel('Loss (in M EUR)')
    plt.title(f'Expected Loss vs. Observed Loss over Time for sample {sample_name}')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# ECL vs OL evaluation
result_table = pd.DataFrame()
merged_X_test = pd.concat([X_test_on,X_test_OOT_full])

for reporting_date in pd.date_range(start='2013-12-01', end='2021-12-31', freq='12M'):
    time_horizons = list(np.arange(0,9))
    corresponding_dates = [reporting_date + relativedelta(years=t) for t in time_horizons]
    corresponding_dates = [date for date in corresponding_dates if date < pd.Timestamp('2022-01-01')]
    time_horizons = np.arange(len(corresponding_dates))
    reporting_dates_list = [reporting_date] * len(time_horizons)
    time_horizons_list = time_horizons.copy()
    corresponding_dates_list = corresponding_dates
    expected_losses = []
    observed_losses = []

    for sample_name, sample in [('Merged', merged_test_oot)]:
        reset_sample = sample.reset_index(drop=True)

        test_survival_probabilities = model.predict_surv_df(merged_X_test.values.astype('float32')).reset_index(drop=True)

        for years_ahead in time_horizons:
            duration_column = reset_sample['duration'] + 12 * years_ahead
            survival = pd.Series(test_survival_probabilities.T.lookup(reset_sample.index, duration_column), index=reset_sample.index)

            df_sample = sample.reset_index(drop=True).copy()
            df_sample['PD'] = 1 - survival

            merged_table = pd.merge(df_sample, df[['cid', 'duration', 'reporting_date', 'Outstanding']], on=['cid', 'duration'], how='left')
            merged_table['EL'] = merged_table['PD'] * merged_table['Outstanding']/1000000

            expected_loss = merged_table.groupby('reporting_date')['EL'].sum().rename('Expected Loss').reset_index()
            expected_loss = expected_loss[expected_loss['reporting_date'] == reporting_date]
            observed_loss = calculate_observed_loss(merged_table, reporting_date, years_ahead)

            expected_losses.extend(expected_loss['Expected Loss'])
            observed_losses.append(observed_loss/1000000)

    data = {
        'Reporting Date': reporting_dates_list,
        'Time Horizon': time_horizons_list,
        'Corresponding Date': corresponding_dates_list,
        'Expected Loss': expected_losses,
        'Observed Loss': observed_losses
    }
    result_table = result_table.append(pd.DataFrame(data), ignore_index=True)

In [ ]:
result_table

In [ ]:
# Iterate through time horizon
for time_horizon in range(9):

    # Plotting the data
    plt.figure(figsize=(10, 6))
    plt.plot(result_table[result_table['Time Horizon']==time_horizon]['Reporting Date'], result_table[result_table['Time Horizon']==time_horizon]['Observed Loss'], label='Observed Loss', marker='o')
    plt.plot(result_table[result_table['Time Horizon']==time_horizon]['Reporting Date'], result_table[result_table['Time Horizon']==time_horizon]['Expected Loss'], label='Expected Loss', marker='o')

    plt.xlabel('Reporting date')
    plt.xticks(result_table[result_table['Time Horizon']==time_horizon]['Reporting Date'].unique())
    plt.ylabel('Loss (in M EUR)')
    plt.title(f'Expected Loss vs. Observed Loss over Time for {time_horizon}-year time horizon OOS and OOT combined')
    plt.legend(loc='upper left')

    plt.grid(True)
    plt.tight_layout()
    plt.show()

### 2.3 Correlation of PD predictions obtained from different models

In [ ]:
#Overall
for sample_name, sample in [('test', test), ('OOT', OOT)]:
    
    #Results
    corr_matrix, p_value, DeepHit_PD, LR_PD, waft_PD, lognormaft_PD, cph_PD = calculate_corr(sample, sample_name)

In [ ]:
# Define the PD models and their corresponding labels
models = [LR_PD, DeepHit_PD, lognormaft_PD, waft_PD, cph_PD]
model_labels = ['Logistic regression PD', 'DeepHit PD', 'Log-Normal AFT PD', 'Weibull AFT', 'Cox PH']

# Iterate through the models and create scatterplots and calculate Spearman coefficients
for i in range(len(models)):
    plt.figure(figsize=(6, 6))
    plt.scatter(OOT['duration'], models[i])
    plt.xlabel('duration')
    plt.ylabel(model_labels[i])
    plt.title(f'Scatterplot between duration of facility in the portfolio and {model_labels[i]} on OOT sample')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    corr_coef, _ = stats.spearmanr(OOT['duration'], models[i])
    print(f"Spearman coefficient between duration of facility in the portfolio and {model_labels[i]} for OOT sample is equal to {corr_coef:.4f}.")

In [ ]:
# Evaluation of AUC values over duration of client in the portfolio
evaluation_results = []
models_to_evaluate = ['Logistic regression', 'Weibull AFT', 'Log-Normal AFT', 'Cox PH']

for sample_name, sample in [('test', test), ('OOT', OOT)]:
    corr_values = []

    for duration in durations_to_evaluate:
        sample_time = sample[sample['duration'] == duration]

        corr_matrix, _, _, _, _, _, _ = calculate_corr(sample_time, sample_name, duration_ind=1)

        for model_name in models_to_evaluate:
            corr_value = corr_matrix[models_to_evaluate.index(model_name), 4]

            # Append correlation values to the list
            evaluation_results.append({
                'Model': model_name,
                'Sample': sample_name,
                'Duration': duration,
                'Corr Value': corr_value
            })

for sample_name in ['OOT', 'test']:
    plt.figure(figsize=(10, 6))
    
    # Iterate through the models and create scatterplots
    for model_name in models_to_evaluate:
        # Filter the evaluation results for the current sample and model
        filtered_results = [item for item in evaluation_results if item['Sample'] == sample_name and item['Model'] == model_name]
        
        # Extract duration and correlation values
        durations = [item['Duration'] for item in filtered_results]
        corr_values = [item['Corr Value'] for item in filtered_results]
        
        # Plot the correlation values for the current model
        plt.plot(durations, corr_values, label=model_name, marker='o')

    plt.xlabel('Duration (months)')
    plt.ylabel('Correlation')
    plt.title(f'Correlation for Different Durations over Time for {sample_name}')
    plt.xticks(durations_to_evaluate)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
#Reset warnings settings to default
warnings.filterwarnings("default")